# K-Nearest Neighbors (KNN) - TensorFlow Implementation

Multi-class classification on the **Covertype (Forest Cover Type)** dataset using TensorFlow tensor operations.

**Dataset**: 581,012 samples, 54 features, 7 forest cover types  
**Task**: Predict forest cover type from cartographic variables  
**Key Concept**: KNN is a "lazy learner" - no training phase, expensive at prediction time

## TensorFlow Approach for KNN
- **Tensor operations**: Broadcasting for pairwise distance computation
- **`tf.math.top_k`**: Efficient K-nearest selection
- **Batched processing**: Memory management for large datasets

## Important Note: CPU-Only Execution
- TensorFlow 2.11+ dropped native Windows GPU support. This implementation runs on CPU.
- For GPU acceleration on Windows, options include WSL2 or TensorFlow 2.10 with Python ≤3.10.
- GPU setup will be configured when we reach neural network models (DNNs, CNNs).


In [3]:
# Standard libraries
import numpy as np
import sys

# TensorFlow for GPU-accelerated tensor operations
import tensorflow as tf

# Add utils to path
sys.path.append('../..')
from utils.data_loader import load_processed_data
from utils.metrics import accuracy, macro_f1_score
from utils.visualization import (
    plot_confusion_matrix_multiclass,
    plot_per_class_f1
)
from utils.performance import track_performance

# Check device
print(f"TensorFlow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU available: {gpus[0].name}")
else:
    print("Running on CPU (TF 2.11+ dropped native Windows GPU support)")

print("Imports complete!")

TensorFlow version: 2.20.0
Running on CPU (TF 2.11+ dropped native Windows GPU support)
Imports complete!
